In [3]:
#import necessary programs
import os
import mne
import numpy as np

# set qt backend for qt notebook plots
# %matplotlib qt

# or for notebook inline plotting
%matplotlib inline


Load and notch filter data

In [4]:
#specify raw file path
DATA_FOLDER = '/Users/logang/Dropbox/GrosenickLab/data/accel_TMS_EEG/MDD/subject6_m160_dlpfc_57/m160_dlpfc_day1/'
FILENAME = 'm160_dlpfc_day1_reststate1_pre_20230327_115800.mff'
data_raw_file = os.path.join(DATA_FOLDER, FILENAME)

#plot the data
raw = mne.io.read_raw_egi(data_raw_file, preload=True)
#raw.plot(n_channels=64, scalings='auto', title='EEG data')

notch_freqs = [60,120,180,240,300,360,400,420,460,480]

def notch_and_hp(raw, notch_freqs, l_freq=1.0, h_freq=None, filter_type='fir'):
    notch_freqs = np.array(notch_freqs)
    raw_notch = raw.copy().notch_filter(freqs=notch_freqs, verbose='warning')
    raw_hp = raw_notch.filter(l_freq=l_freq, h_freq=h_freq, method=filter_type, verbose='warning')
    return raw_hp

raw = notch_and_hp(raw, notch_freqs=notch_freqs)
raw.set_eeg_reference("average")


Reading EGI MFF Header from /Users/logang/Dropbox/GrosenickLab/data/accel_TMS_EEG/MDD/subject6_m160_dlpfc_57/m160_dlpfc_day1/m160_dlpfc_day1_reststate1_pre_20230327_115800.mff...
    Reading events ...
    Assembling measurement info ...
Reading 0 ... 180555  =      0.000 ...   180.555 secs...
EEG channel type selected for re-referencing
Applying average reference.
Applying a custom ('EEG',) reference.


<RawMff | signal1.bin, 257 x 180556 (180.6 s), ~354.3 MB, data loaded>

Autoreject bad segments

In [5]:
import numpy as np
from autoreject import AutoReject, Ransac

# generate epochs object from raw
epochs = mne.make_fixed_length_epochs(raw, duration=1.0, preload=True)
print(epochs.get_data().shape)


n_interpolates = np.array([1, 4, 32])
consensus_percs = np.linspace(0, 1.0, 11)
ar = AutoReject(n_interpolates, consensus_percs, thresh_method='bayesian_optimization', random_state=42)
epochs = ar.fit_transform(epochs)

# plot results
ar.get_reject_log(epochs).plot()


Not setting metadata
180 matching events found
No baseline correction applied
0 projection items activated
Using data from preloaded Raw for 180 events and 1000 original time points ...
0 bad epochs dropped
(180, 257, 1000)
Running autoreject on ch_type=eeg


  0%|          | Creating augmented epochs : 0/257 [00:00<?,       ?it/s]

  0%|          | Computing thresholds ... : 0/257 [00:00<?,       ?it/s]

KeyboardInterrupt: 

In [ ]:
?ar.get_reject_log(epochs).plot()


Check continuity and some segment boundaries and replace raw data to have continuous cleaned raw object

In [4]:
import matplotlib.pyplot as plt
X = np.concatenate(epochs.get_data(), axis=1)
plt.plot(X[2,900:1100]) # spot check continuity 
plt.plot(X[2,8900:9100]) # spot check continuity 
plt.show()

new_raw = mne.io.RawArray(X,raw.info)


Creating RawArray with float64 data, n_channels=257, n_times=149000
    Range : 0 ... 148999 =      0.000 ...   148.999 secs
Ready.


Remove bad channels using NoisyChannels from pyprep

In [5]:
from pyprep.find_noisy_channels import NoisyChannels
nc = NoisyChannels(new_raw, random_state=42)
nc.find_all_bads(channel_wise=True)

bad_channels = nc.bad_by_deviation + nc.bad_by_hf_noise + nc.bad_by_dropout + nc.bad_by_ransac + nc.bad_by_correlation

print("Bad by deviation:", nc.bad_by_deviation)
print("Bad by high frequency noise:",nc.bad_by_hf_noise)
print("Bad by dropout:",nc.bad_by_dropout) 
print("Bad by ransac:",nc.bad_by_ransac)
print("Bad by correlation:",nc.bad_by_correlation)
print("Total bads:", len(bad_channels))

raw.info['bads'].extend(bad_channels)
raw.interpolate_bads(reset_bads=True) # This will clear out data.info['bads']
raw.info['bads'] = ['VREF'] # Reset data.info['bads'] to always bad channels
print("Data shape post interpolation:", new_raw._data.shape)


NOTE: pick_types() is a legacy function. New code should use inst.pick(...).
Setting up high-pass filter at 1 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal highpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 1.00
- Lower transition bandwidth: 1.00 Hz (-6 dB cutoff frequency: 0.50 Hz)
- Filter length: 3301 samples (3.301 s)



[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    0.0s
[Parallel(n_jobs=1)]: Done  71 tasks      | elapsed:    0.1s
[Parallel(n_jobs=1)]: Done 161 tasks      | elapsed:    0.4s


Executing RANSAC
This may take a while, so be patient...
Finding optimal chunk size : 123
Total # of chunks: 2
Current chunk: 1 2 
RANSAC done!
Bad by deviation: ['E20', 'E22', 'E25', 'E26', 'E27', 'E31', 'E32', 'E37', 'E38', 'E39']
Bad by high frequency noise: []
Bad by dropout: []
Bad by ransac: ['E1', 'E134', 'E187', 'E191', 'E220', 'E253']
Bad by correlation: ['E25', 'E31', 'E32', 'E37', 'E54']
Total bads: 21
Interpolating bad channels
    Automatic origin fit: head of radius 96.5 mm
Computing interpolation matrix from 240 sensor positions
Interpolating 17 sensors
Data shape post interpolation: (257, 149000)


In [11]:
from mne.preprocessing import ICA
from mne_icalabel import label_components

save_file = '/Users/logang/Dropbox/GrosenickLab/Code/CLEAN/results/ICALabel_ICA_topoplots.png'

def iclabel(mne_raw, num_components=20, keep=["brain"], method='infomax', fit_params=dict(extended=True), reject=None):
    '''
    Uses the MNE-ICALabel method to classify independent components into six different estimated sources of 
    variability: brain, muscle artifact, eye blink, heart beat, line noise, channel noise, and other.
    Args:
        mne_raw: a data file in the MNE-Python raw format.
        n_components: the number of ICA components to estimate. 
        method: the ICA method to use, see MNE documentation for more information.
        fit_params: additional arguments passed to the ICA method. 
        keep: a list of source types to keep; 
              options are 'brain', 'muscle artifact', 'eye blink', 
              'heart beat', 'line noise', 'channel noise', and 'other'.
    Returns:
        ic_labels: a set of labels for the ICs estimated.
        ica: a fit MNE ICA object.
        cleaned: a new data file with the all but the data categories in 'keep' removed.
    '''
    # Band pass data to band consistent with mne-icalabel training data
    mne_filt = mne_raw.filter(l_freq=1.0, h_freq=100.0, verbose='warning')

    # Fit ICA to mne_raw
    print('  Fitting ICA with '+str(num_components)+' components.')
    ica = ICA(n_components=num_components, max_iter="auto", method=method, random_state=42, fit_params=fit_params)
    ica.fit(mne_filt, reject=reject, start=0, stop=mne_filt._data.shape[1])
    
    # Use label_components function from ICLabel library to classify ICs and get their estimated probabilities
    print('  Using ICALabel to classify independent components.')
    ic_labels = label_components(mne_filt, ica, method="iclabel")
    labels = ic_labels["labels"]
    
    # Exclude ICs not in 'keep' list and reconstruct cleaned raw data
    exclude_idx = [idx for idx, label in enumerate(labels) if label not in keep]
    print(f"  Excluding these ICA components: {exclude_idx}")
    cleaned = ica.apply(mne_raw, exclude=exclude_idx, verbose=False)
    return ic_labels, ica, cleaned

def save_ica_components(save_file, ica, ic_label_obj=None):
    '''
    Save a set of independent component plots using MNE-Python's plot_components method for ICA objects.
    Args:
        save_file: absolute path to save PNG output file.
        ica: an MNE-Python ica object.
        ic_label_list: optional list of conformal labels from ica_label to add to plot topomap titles.
    
    '''
    num_components = ica.n_components
    if ic_label_obj is not None:
        ic_label_list = ic_label_obj['labels']
        ic_label_probs = np.round(100*ic_label_obj['y_pred_proba'])
    # Iterate over batches of 20 ICs, making a plot for each group and adding labels and label probabilities from ICALabel
    for i in range(int(num_components/20)+1):
        if (i+1)*20 < num_components:
            picks = list(range(i*20,(i+1)*20))
            if len(picks) > 0:  
                fig = ica.plot_components(picks=picks, show=False)
        else:
            picks = list(range(i*20,num_components))
            if len(picks) > 0:
                fig = ica.plot_components(picks=picks, show=False)
        if ic_label_obj is not None:
            if len(picks) > 0:
                ic_label_list_part = ic_label_list[(i*20):((i+1)*20)]
                ic_probs_list_part = ic_label_probs[(i*20):((i+1)*20)]
                axs = fig.get_axes()
                for j,ax in enumerate(axs):
                    current_title = ax.get_title()
                    ax.set_title(current_title+':\n '+ic_label_list_part[j]+' ('+str(ic_probs_list_part[j])+'%)')
            plt.savefig(save_file.split('.')[0]+'_'+str(i).zfill(2)+save_file.split('.')[1])
            plt.close()

ic_labels, ic_ica, ic_cleaned = iclabel(new_raw, num_components=10)

# Save ICA topoplots and a folder of individual IC statistic plots
#save_ica_components(save_file, ic_ica, ic_label_obj=ic_labels)

#         ic_save_path = pjoin(self.results_savepath,'ICALabel_ICA_topoplots/')
#         if not os.path.exists(ic_save_path):
#            os.makedirs(ic_save_path)
#         for i,label in enumerate(self.ic_labels['labels']):
#             save_single_ic_plot(pjoin(ic_save_path,'IC'+str(i).zfill(3)+'.png'), ic_ica, self.data, component_number=i, ic_label_obj=self.ic_labels)
#             plt.close()

# Generate artifact plots for cleaned data 
#save_eog_plot(pjoin(self.results_savepath,'eog_icalabel_cleaned.png'), ic_cleaned, raw=False)

# Generate histogram of icalabel prediction probabilities
#save_icalabel_prob_hist(pjoin(self.results_savepath,'icalabel_probability_histogram.png'), self.ic_labels)

# Replace pipeline data with new cleaned data
#self.data = ic_cleaned



  Fitting ICA with 10 components.
Fitting ICA to data using 257 channels (please be patient, this may take a while)
Selecting by number: 10 components
Computing Extended Infomax ICA
Fitting ICA took 7.1s.
  Using ICALabel to classify independent components.
NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).
  Excluding these ICA components: [2, 5, 7, 8, 9]


In [12]:
save_ica_components(save_file, ic_ica, ic_label_obj=ic_labels)
